# 1.3 Retrieval

In this notebook we will use a toy search to retrieve data from the FAQs for the purpose of providing context to the LLM down the line.

In [2]:
import minsearch
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Indexing the documents

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
index.fit(documents)

## Searching the documents using Minsearch

In [10]:
q = 'The course has already started, can I still enroll?'

In [ ]:
#boost allows you to dictate the importance of fields when looking for search results.
boost = {
    'question': 3.0,
    'section': 0.5,
}

results = index.search(
    query=q,
    boost_dict=boost,
    filter_dict={'course': 'data-engineering-zoomcamp'}, # limit search to a particular course
    num_results=5
)

In [18]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# 1.4 Generation with OpenAI

## Config connection to OpenAI

In [21]:
from openai import OpenAI
import os
from dotenv import dotenv_values


In [22]:
secrets_dir = os.path.expanduser("~/Desktop/projects/.secrets")

config = {
    **dotenv_values(secrets_dir + "/.env.llm-zc"),
}

OPENAI_KEY = config.get("OPEN_AI_API_KEY")

In [23]:
client = OpenAI(
    api_key=OPENAI_KEY
)

In [24]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [ ]:
response.choices[0].message.content #gives a generic response as it does not have context yet.

"To determine if you can still enroll in a course that has already begun, you should consider the following steps:\n\n1. **Check the Course Policy:** Review any enrollment guidelines provided by the institution or course organizer. Some courses may allow late enrollment, especially if they're online or self-paced.\n\n2. **Contact the Instructor or Institution:** Reach out directly to the course instructor or the admissions office. Explain your situation and inquire about the possibility of late enrollment.\n\n3. **Consider Prerequisites and Catch-up:** If late enrollment is possible, assess whether you can quickly catch up on missed material or meet any prerequisites.\n\n4. **Understand the Implications:** Be aware of any potential downsides to enrolling late, such as missing important introductory sessions or a more challenging workload to catch up.\n\n5. **Check for Fees or Penalties:** Confirm if there are any additional fees or penalties associated with late enrollment.\n\nTaking t

Now we will use the responses from the knowledge base to get more specific response from open ai.

In [28]:
prompt_template = """
You're a course teaching assistant. You support prospective and current students, by answering questions they have about the courses we offer.
Answer the QUESTION based on the CONTEXT provided from our FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
"""

In [31]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [ ]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [34]:
prompt = prompt_template.format(question=q, context=context).strip()

In [35]:
print(prompt)

You're a course teaching assistant. You support prospective and current students, by answering questions they have about the courses we offer.
Answer the QUESTION based on the CONTEXT provided from our FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: The course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also contin

In [36]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register before the course start date, you are still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so it's important not to leave everything for the last minute."